# Initial import

In [ ]:
import numpy as np
from permeatus import *

# layered1D module

## 2-layer problem, as in Macher et al. paper (https://doi.org/10.1016/j.ijhydene.2021.04.095)

Instantiate layered1D class with input parameters

In [ ]:
perm = layered1D(materials=2,L=[0.5,0.5],D=[1.0,0.1],S=[1.0,1.1],\
              C0=1.0,C1=0.0,touts=[0.001,0.05,0.2,2.0],tstep=0.001,ncpu=8)

Create layered mesh with Gmsh, and write Abaqus input file

In [ ]:
perm.layered_mesh(N=[50,50])

Run Abaqus job, with output for concentration, flux, and integration point volume, outputted in .csv files

In [ ]:
perm.submit_job()

Read concentration and flux fields into Python dictionary

In [ ]:
perm.read_field('C')

In [ ]:
perm.read_field('J')

Print field dictionary to show data structure

In [ ]:
print(perm.field)

Calculate mean and standard deviation of the y-component of flux, which is in the direction of the concentration gradient

In [ ]:
print(np.mean(perm.field['J'][-1]['data'][:,1]))
print(np.std(perm.field['J'][-1]['data'][:,1]))

Plot concentration results, along with the analytical steady state solution

In [ ]:
perm.plot_1d(plotTarget='C',showPlot=False)
x, xc, p, C, J = perm.steady_state(plot=True,plotTarget='C')

Plot pressure results similarly

In [ ]:
perm.plot_1d(plotTarget='p',showPlot=False)
x, xc, p, C, J = perm.steady_state(plot=True,plotTarget='p')

Get effective properties by numerical averaging, and by analytical solution (Reuss bound)

In [ ]:
perm.get_eff_coeffs('numerical')
print(perm.D_eff,perm.S_eff,perm.P_eff)

In [ ]:
perm.get_eff_coeffs('analytical')
print(perm.D_eff,perm.S_eff,perm.P_eff)

# Permeatus n-layer planar, with pressure formulation

If only permeation coefficients are known, then problems can be specified in this alternative way

In [ ]:
perm = layered1D(materials=3,L=[0.4,0.3,0.3],P=[1.0,0.11,0.35],\
              p0=1.0,p1=0.0,touts=[0.001,0.05,0.2,2.0],tstep=0.001,ncpu=8,verbose=False)

Everything else proceeds as before, but only the pressure solution is known

In [ ]:
perm.layered_mesh(N=[40,30,30])

In [ ]:
perm.submit_job()

In [ ]:
perm.read_field('C')

In [ ]:
perm.read_field('J')

In [ ]:
print(np.mean(perm.field['J'][-1]['data'][:,1]))
print(np.std(perm.field['J'][-1]['data'][:,1]))

In [ ]:
perm.plot_1d(plotTarget='p',showPlot=False)
x, xc, p, C, J = perm.steady_state(plot=True,plotTarget='p')

In [ ]:
perm.get_eff_coeffs('numerical')
print(perm.P_eff)

In [ ]:
perm.get_eff_coeffs('analytical')
print(perm.P_eff)

# homogenisation module

This class is similar to the previous one, but now layer thicknesses are not specified, and material volume fractions are specified

In [ ]:
perm = homogenisation(materials=2,D=[1.0,0.1],S=[1.0,1.1],vFrac=[0.4,0.6],\
              C0=1.0,C1=0.0,touts=[5,10],tstep=0.001,ncpu=8)

## Reuss Mesh

Create a serially layered system whose analytical solution is the Reuss bound

In [ ]:
perm.reuss_mesh(Nx=1,Ny=100)

In [ ]:
perm.submit_job()

Get effective properties by numerical averaging

In [ ]:
perm.get_eff_coeffs()
print(perm.D_eff,perm.S_eff,perm.P_eff)

Compare with analytical Reuss bound

In [ ]:
perm.reuss_bound()
print(perm.D_eff,perm.S_eff,perm.P_eff)

## Voigt Mesh

Create a parallel layered system whose analytical solution is the Voigt bound

In [ ]:
perm.voigt_mesh(Nx=20,Ny=80)

In [ ]:
perm.submit_job()

Compare numerical and analytical results for effective properties

In [ ]:
perm.get_eff_coeffs()
print(perm.D_eff,perm.S_eff,perm.P_eff)

In [ ]:
perm.voigt_bound()
print(perm.D_eff,perm.S_eff,perm.P_eff)

## Fibre-reinforced composite 2D cross section

In [ ]:
perm.cross_section_mesh(nc=15,r=0.1,algorithm='LS')

In [ ]:
perm.submit_job()

Compare numerical averaging with numerous analytical bounds and approximations

In [ ]:
perm.get_eff_coeffs()
print(perm.D_eff,perm.S_eff,perm.P_eff)

In [ ]:
perm.voigt_bound()
print(perm.D_eff,perm.S_eff,perm.P_eff)

In [ ]:
perm.reuss_bound()
print(perm.D_eff,perm.S_eff,perm.P_eff)

In [ ]:
perm.HS_upper_bound()
print(perm.D_eff,perm.S_eff,perm.P_eff)

In [ ]:
perm.HS_lower_bound()
print(perm.D_eff,perm.S_eff,perm.P_eff)

In [ ]:
perm.maxwell_eucken()
print(perm.D_eff,perm.S_eff,perm.P_eff)

In [ ]:
perm.bruggeman()
print(perm.D_eff,perm.S_eff,perm.P_eff)

In [ ]:
perm.chen()
print(perm.D_eff,perm.S_eff,perm.P_eff)

For Nielsen model, the aspect ratio of the dispersed phase must also be specified

In [ ]:
perm.AR = 1
perm.nielsen()
print(perm.D_eff,perm.S_eff,perm.P_eff)